In [189]:
import urllib
import requests
import json
from datetime import datetime
from datetime import date


def download_weather_climate_division(county: str, date_to_start: str):
    base_url = "http://data.rcc-acis.org/GridData"
    start_date = datetime.strptime(date_to_start, '%Y%m%d').strftime("%Y%m%d")
    end_date = datetime.today().strftime('%Y%m%d')
        
    
    if county.lower() == 'cook':
        fips = '17031'
    elif county.lower() == 'dupage' or county.lower() == 'du page':
        fips = '17043'
    else:
        raise ValueError("Must be either Cook or DuPage county.")
    
#     params_n = json.dumps({"county":fips,
#                      "sdate":start_date,
#                      "edate":end_date,
#                      "grid":21,
#                      "elems":[{"name":"mint","area_reduce":"county_mean"}]})
    
    params = {"state":"IL","grid":"21","date":"2012","elems":[{"name":"yly_pcpn","area_reduce":"climdiv_mean"}]}
    data = urllib.parse.urlencode(params)
    print(data)
    headers = {"Content-Type": "application/json"}
    req = urllib.request.Request(base_url, data, headers)
    print(req.data)
    
    with urllib.request.urlopen(req) as response:
        the_page = response.read()
        
    print(the_page)
    
#     n_json = urllib.request.urlopen(full_url)
#     print(n_json)
#     n_json = json.loads(n_json.read())
#     print(n_json)

In [190]:
download_weather_climate_division('cook', '20100101')

state=IL&grid=21&date=2012&elems=%5B%7B%27name%27%3A+%27yly_pcpn%27%2C+%27area_reduce%27%3A+%27climdiv_mean%27%7D%5D
state=IL&grid=21&date=2012&elems=%5B%7B%27name%27%3A+%27yly_pcpn%27%2C+%27area_reduce%27%3A+%27climdiv_mean%27%7D%5D


TypeError: POST data should be bytes, an iterable of bytes, or a file object. It cannot be of type str.

In [ ]:
# request data
params_n = json.dumps({"state":"IL",
                     "sdate":start_date,
                     "edate":end_date,
                     "grid":21,
                     "elems":[{"name":"mint","area_reduce":"climdiv_mean"}]})

req_n = urllib3.request(config['prism_url'],params_n,{"Content-Type":"application/json"})
response = urllib3.urlopen(req_n)
n_json = json.loads(response.read())['data']


params_x = json.dumps({"state":"IL",
                     "sdate":start_date,
                     "edate":end_date,
                     "grid":21,
                     "elems":[{"name":"maxt","area_reduce":"climdiv_mean"}]})
req_x = urllib3.request(config['prism_url'],params_x,{"Content-Type":"application/json"})
response = urllib3.urlopen(req_x)
x_json = json.loads(response.read())['data']

params_p = json.dumps({"state":"IL",
                     "sdate":start_date,
                     "edate":end_date,
                     "grid":21,
                     "elems":[{"name":"pcpn","area_reduce":"climdiv_mean"}]})

req_p = urllib3.request(config['prism_url'],params_p,{"Content-Type":"application/json"})
response = urllib3.urlopen(req_p)
p_json = json.loads(response.read())['data']

In [5]:
for i in range(1,10):
    file = open("../output_files/110"+str(i),"w+")
    for n,x,p in zip(n_json,x_json,p_json):
        ymd = n[0].split("-")
        if bool(n[1]) and bool(x[1]) and bool(p[1]):
            file.write("IL" + "\t" + str(i) + "\t" + ymd[0] + "\t"  + ymd[1].lstrip("0") + "\t" + ymd[2].lstrip("0") + "\t" + str(float("{0:.1f}".format(n[1]["IL0"+str(i)]))) + "\t" + str(float("{0:.1f}".format(x[1]["IL0"+str(i)]))) + "\t" + str(float("{0:.2f}".format(p[1]["IL0"+str(i)]))) + "\n")
    file.close()

/home/jallen17/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  


TypeError: 'module' object is not callable